In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

import mediapipe as mp # Import mediapipe
import cv2 # Import opencv


mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

import csv
import os
import numpy as np
import pandas as pd

import pickle 

In [2]:
with open('yoga_pose_detector.pkl', 'rb') as f:
    model = pickle.load(f)

In [3]:
model

NameError: name 'results' is not defined

In [16]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        pose_hands = [15, 16, 17, 18, 19, 20, 21, 22]
        
        if results.right_hand_landmarks:
            right_hand = results.right_hand_landmarks.landmark
            
            #print('\nright hand:\n', right_hand)
        
        if results.left_hand_landmarks:
            left_hand = results.left_hand_landmarks.landmark
            #print('\nleft hand:\n', left_hand)
            
        if results.pose_landmarks:
            pose = np.array(results.pose_landmarks.landmark)
            pose[pose_hands][:] = np.NaN
            print('\npose:\n', pose)
        
        
        
        
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
                                 )

        # 4. Pose Detections
        if not results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
                                 )
        else:
            mp_drawing.draw_landmarks(image, pose, mp_holistic.POSE_CONNECTIONS, 
                                 landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
                                 )
            
            
        # Export coordinates
        try:
            # 1. Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Concate rows
            row = np.array(pose_row)

            # Make Detections
            X = row.reshape(1, -1)
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0] - 0.05
            
            rightEarX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].x
            rightEarY = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].y
            leftFootX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].x
            leftFootY = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].y
            
#             cv2.rectangle(image, 
#                           (leftEarX, leftEarY), 
#                           (leftFootX, leftFootY), 
#                           (245, 117, 16), -1)
#             print('rectangle works')
#             cv2.putText(image, body_language_class, coords, 
#                         cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
#             print('text works')
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            if(round(body_language_prob[np.argmax(body_language_prob)],2) > 0.8):
                # Display Class
                cv2.putText(image, 'CLASS'
                            , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, body_language_class.split(' ')[0]
                            , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display Probability
                cv2.putText(image, 'PROB'
                            , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                            , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            else:
                # Display Class
                cv2.putText(image, 'CLASS'
                            , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, 'N/A'
                            , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display Probability
                cv2.putText(image, 'PROB'
                            , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, 'N/A'
                            , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
            
        except:
            print("except")
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()




pose:
 [x: 0.7969011664390564
 y: 0.6539226770401001
 z: -1.1456778049468994
 visibility: 0.9872105121612549 x: 0.8350991010665894
                                y: 0.5957056283950806
                                z: -1.0961625576019287
                                visibility: 0.9704867601394653
 x: 0.8557632565498352
 y: 0.6007295250892639
 z: -1.09661066532135
 visibility: 0.9717845916748047 x: 0.8730222582817078
                                y: 0.605803370475769
                                z: -1.097451090812683
                                visibility: 0.9678177833557129
 x: 0.7704367637634277
 y: 0.5884039402008057
 z: -1.095063328742981
 visibility: 0.9804283976554871 x: 0.7500760555267334
                                y: 0.5879299640655518
                                z: -1.0947695970535278
                                visibility: 0.9834691286087036
 x: 0.731171727180481
 y: 0.5891975164413452
 z: -1.0955547094345093
 visibility: 0.9828634858131409 x: 0.894

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [25]:
print(results.left_hand_landmarks.landmark[0])

AttributeError: 'NoneType' object has no attribute 'landmark'

In [23]:
row = row.reshape(1, -1)
row.shape
row[0]

array([ 0.49928629,  0.21159273, -0.29513225,  0.99999833,  0.50446355,
        0.20154771, -0.28046381,  0.99999255,  0.50768173,  0.2023067 ,
       -0.28048283,  0.99999356,  0.51072919,  0.20293786, -0.28049281,
        0.99999356,  0.49548054,  0.2010152 , -0.28098184,  0.99998999,
        0.49261877,  0.20128243, -0.28099647,  0.99999052,  0.48985609,
        0.20141524, -0.28096443,  0.99998897,  0.51563096,  0.21397486,
       -0.18057965,  0.99998754,  0.486388  ,  0.21357663, -0.18405944,
        0.99997157,  0.50655258,  0.22787748, -0.25714299,  0.99999624,
        0.49440128,  0.22675349, -0.25822312,  0.9999938 ,  0.54344392,
        0.3184669 , -0.10720768,  0.99999297,  0.46057889,  0.32589585,
       -0.10165274,  0.99995321,  0.56327075,  0.42508832, -0.05732214,
        0.99660546,  0.44164038,  0.43399385, -0.05125147,  0.98569781,
        0.59457189,  0.51421148, -0.14749977,  0.99577802,  0.41170785,
        0.52941018, -0.13626719,  0.97997403,  0.60723191,  0.54

In [19]:
model.predict(row)


array(['Mountain Fold'], dtype=object)

In [29]:
model.predict_proba(row)[0][0]

0.0

In [ ]:
pose:
 [x: 0.7969011664390564
 y: 0.6539226770401001
 z: -1.1456778049468994
 visibility: 0.9872105121612549 x: 0.8350991010665894
                                y: 0.5957056283950806
                                z: -1.0961625576019287
                                visibility: 0.9704867601394653
 x: 0.8557632565498352
 y: 0.6007295250892639
 z: -1.09661066532135
 visibility: 0.9717845916748047 x: 0.8730222582817078
                                y: 0.605803370475769
                                z: -1.097451090812683
                                visibility: 0.9678177833557129
 x: 0.7704367637634277
 y: 0.5884039402008057
 z: -1.095063328742981
 visibility: 0.9804283976554871 x: 0.7500760555267334
                                y: 0.5879299640655518
                                z: -1.0947695970535278
                                visibility: 0.9834691286087036
 x: 0.731171727180481
 y: 0.5891975164413452
 z: -1.0955547094345093
 visibility: 0.9828634858131409 x: 0.894318699836731
                                y: 0.6566526889801025
                                z: -0.7036793231964111
                                visibility: 0.9740489721298218
 x: 0.693313717842102
 y: 0.6256954669952393
 z: -0.6569032073020935
 visibility: 0.990128219127655 x: 0.8227123022079468
                               y: 0.7516559362411499
                               z: -0.9900447726249695
                               visibility: 0.994056224822998
 x: 0.7442919015884399
 y: 0.7352864742279053
 z: -0.9779956936836243
 visibility: 0.9955981373786926 x: 0.9628898501396179
                                y: 1.024590015411377
                                z: -0.37880340218544006
                                visibility: 0.9893494844436646
 x: 0.5160354375839233
 y: 0.9266648888587952
 z: -0.4897025227546692
 visibility: 0.9969416260719299 x: 0.9914180636405945
                                y: 1.5033073425292969
                                z: -0.21079280972480774
                                visibility: 0.08833245933055878
 x: 0.27867889404296875
 y: 1.2667430639266968
 z: -0.6848124265670776
 visibility: 0.3145349621772766 x: 0.8978177309036255
                                y: 1.793968677520752
                                z: -0.4760715961456299
                                visibility: 0.007855855859816074
 x: 0.24964383244514465
 y: 1.5783740282058716
 z: -1.4106024503707886
 visibility: 0.03401735797524452 x: 0.8893736004829407
                                 y: 1.9077645540237427
                                 z: -0.5616617202758789
                                 visibility: 0.008303569629788399
 x: 0.20915694534778595
 y: 1.6899536848068237
 z: -1.5774532556533813
 visibility: 0.02569320984184742 x: 0.854718029499054
                                 y: 1.896379828453064
                                 z: -0.6344817876815796
                                 visibility: 0.0129520483314991
 x: 0.26096221804618835
 y: 1.6952415704727173
 z: -1.635749101638794
 visibility: 0.03372660651803017 x: 0.8484748601913452
                                 y: 1.8497227430343628
                                 z: -0.5188583731651306
                                 visibility: 0.01324846688657999
 x: 0.2837722599506378
 y: 1.655457854270935
 z: -1.4544707536697388
 visibility: 0.03491739183664322 x: 0.7254335880279541
                                 y: 1.7610828876495361
                                 z: -0.061301589012145996
                                 visibility: 0.004327019210904837
 x: 0.4374074935913086
 y: 1.6376030445098877
 z: 0.06494902074337006
 visibility: 0.005874938797205687 x: 0.5665457844734192
                                  y: 2.344316244125366
                                  z: 0.21271996200084686
                                  visibility: 0.0005808992427773774
 x: 0.30309030413627625
 y: 2.231733798980713
 z: 0.6140896677970886
 visibility: 0.0002786508703138679 x: 0.4141736626625061
                                   y: 2.8466341495513916
                                   z: 1.1495901346206665
                                   visibility: 0.00017928182205650955
 x: 0.15949036180973053
 y: 2.7365646362304688
 z: 1.6742194890975952
 visibility: 6.12759540672414e-05 x: 0.3953276574611664
                                  y: 2.928906202316284
                                  z: 1.2108889818191528
                                  visibility: 0.00013597772340290248
 x: 0.13062593340873718
 y: 2.8159537315368652
 z: 1.7690155506134033
 visibility: 0.00021849689073860645 x: 0.32377731800079346
                                    y: 3.0004849433898926
                                    z: 0.5747854709625244
                                    visibility: 0.00011323222133796662
 x: 0.15471407771110535
 y: 2.9384384155273438
 z: 1.1051139831542969
 visibility: 0.00015874183736741543]